# validate extracting fgrid file to numpy file

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [1]:
from fgrid2npy_JD import extract_fgrid_coordinates
coor_npy = extract_fgrid_coordinates(
    fgrid_file_path = "data/JD_Sula_2025_flow.fgrid",
    save_dir = "results/fgrid_coordinates.npy"
    )

Parsing the fgrid file: data/JD_Sula_2025_flow.fgrid
Units: METRES  
Processing 989001 coordinate sections...
Total cells processed: 989001
Cells filled: 989001/989001 (100.0%)
Coordinates array shape: (107, 117, 79, 3)
Coordinates array dtype: float64
Coordinate Statistics:
  X range: 424730.20 to 477843.57
  Y range: 7072225.88 to 7129985.88
  Z range: 348.63 to 2865.07
Data saved to results/fgrid_coordinates.npy


In [2]:
print(coor_npy[63,107,42,0])
print(coor_npy[63,107,42,1])
print(coor_npy[63,107,42,2])

456184.7025
7076741.875
923.88880875


In [5]:
# data copied directly from the fgrid file
data_string = """
    4.5594269E+05    7.0769925E+06    9.1917834E+02    4.5643500E+05
    7.0769920E+06    9.1349969E+02    4.5593431E+05    7.0764915E+06
    9.2084918E+02    4.5642681E+05    7.0764915E+06    9.1518518E+02
    4.5594269E+05    7.0769925E+06    9.3266486E+02    4.5643500E+05
    7.0769920E+06    9.2686707E+02    4.5593431E+05    7.0764915E+06
    9.3432288E+02    4.5642681E+05    7.0764915E+06    9.2854327E+02
"""

flat_array = np.fromstring(data_string, sep=' ')
print(flat_array.shape)
print(flat_array[0::3].mean())
print(flat_array[1::3].mean())
print(flat_array[2::3].mean())

(24,)
456184.7025
7076741.875
923.88880875


# validate fault id by checking against Petrel fault property

In [ ]:
import numpy as np
fault_npy = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
# fault_npy[94,43,40,3]
fault_npy[50,117-1-25,40,3]

7.0

# validate fault slip analysis

## compute one point using geothermal fault slip code to check if matrix based 3d stress transofrmation and fault slip analysis are correct

In [2]:
# from geothermal fault slip analysis
import numpy as np

def StressTransform3D(Pf,SH,Sh,Sv,phi,theta):
    # set up principle stress matrix
    s = np.array([[SH-Pf,0,0],[0,Sh-Pf,0],[0,0,Sv-Pf]])
    # pre-calculate trigonometric values
    cos_phi = np.cos(np.radians(phi))
    sin_phi = np.sin(np.radians(phi))
    cos_theta = np.cos(np.radians(theta))
    sin_theta = np.sin(np.radians(theta))
    # perform stress tranformation
    # first rotate around z axis (vertical stress direction)
    Rz = np.array([[cos_phi,-sin_phi,0],[sin_phi,cos_phi,0],[0,0,1]])
    # next rotate around x axis (maximum horizotnal stress direction)
    Rx = np.array([[1,0,0],[0,cos_theta,-sin_theta],[0,sin_theta,cos_theta]])
    # compute rotation matrix using rotation components in reverse order
    R = Rx @ Rz
    # perform rotation
    result = R @ s @ (R.T)
    # retrieve stresses from matrices
    tau1 = result[2,0] # shear stress component in the strike direction
    tau2 = result[2,1] # shear stress component in the dip direction
    tau = (result[2,0]**2 + result[2,1]**2)**0.5 # shear stress
    sigma = result[2,2] # normal stress

    return sigma, tau

# compute fault slip indicator for a single point
# input stress state
SH_grad = 15.25 #maximum horizontal stress gradient in MPa/km
Sh_grad = 4.89 #minimun horizontal stress gradient in MPa/km
Sv_grad = 11.69 #vertical stress gradient in MPa/km
SH_azi = 292.13 #maximum horizontal stress direction in degree, relative to north clockwise, range [0,180]
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

fault_strike = 10
fault_dip = 90

# load data
coor_fault = np.load('results/JD_Sula_2025_flow_coor&fault.npy')
pres = np.load('results/case1_PRES.npy')
# extract coordinates
x_coor = coor_fault[20,40,60,0]
y_coor = coor_fault[20,40,60,1]
z_coor = coor_fault[20,40,60,2]
# compute principal stresses
SH_stress = z_coor /1000 * SH_grad
Sh_stress = z_coor /1000 * Sh_grad
Sv_stress = z_coor /1000 * Sv_grad
# extract pressure
pres_slice = pres[20,40,60,2]/1000 # convert to MPa
# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip
# transform principla stresses to fault planes
sigma, tau = StressTransform3D(pres_slice, SH_stress, Sh_stress, Sv_stress, phi, theta)
# compute fault slip indicator, where 1 indicates slip and 0 indicates stability
fault_slip = ((tau - cohesion) / sigma >= mu).astype(int)
print(sigma, tau)
print(fault_slip)

6.620857434092926 4.5839854638042565
0


# validate extracted stresses match with values read from CMG Results software

In [6]:
import numpy as np
SH = np.load('data/validation/250819_stresses/case1_STRESMXP.npy')
Sh = np.load('data/validation/250819_stresses/case1_STRESMNP.npy')
Sv = np.load('data/validation/250819_stresses/case1_STRESINT.npy')
print(SH.shape)

i = 41; j = 47; k = 6 # a fault cell
print(SH[i-1,j-1,k-6,0])
print(Sh[i-1,j-1,k-6,0])
print(Sv[i-1,j-1,k-6,0])
# checked the above matched exactly

(107, 117, 5, 6)
22267.4
7093.82
17930.8


# validate fault slip analysis stress based

checked 3 cells that all match

In [1]:
# stress transformation for single values
def StressTransform3D(Pf,SH,Sh,Sv,phi,theta):
    # set up principle stress matrix
    s = np.array([[SH-Pf,0,0],[0,Sh-Pf,0],[0,0,Sv-Pf]])
    # pre-calculate trigonometric values
    cos_phi = np.cos(np.radians(phi))
    sin_phi = np.sin(np.radians(phi))
    cos_theta = np.cos(np.radians(theta))
    sin_theta = np.sin(np.radians(theta))
    # perform stress tranformation
    # first rotate around z axis (vertical stress direction)
    Rz = np.array([[cos_phi,-sin_phi,0],[sin_phi,cos_phi,0],[0,0,1]])
    # next rotate around x axis (maximum horizotnal stress direction)
    Rx = np.array([[1,0,0],[0,cos_theta,-sin_theta],[0,sin_theta,cos_theta]])
    # compute rotation matrix using rotation components in reverse order
    R = Rx @ Rz
    # perform rotation
    result = R @ s @ (R.T)
    # retrieve stresses from matrices
    tau1 = result[2,0] # shear stress component in the strike direction
    tau2 = result[2,1] # shear stress component in the dip direction
    tau = (result[2,0]**2 + result[2,1]**2)**0.5 # shear stress
    sigma = result[2,2] # normal stress

    return sigma, tau

In [19]:
import numpy as np
SH = np.load('data/validation/FSA_stress_based/case1_STRESMXP.npy')
Sh = np.load('data/validation/FSA_stress_based/case1_STRESMNP.npy')
Sv = np.load('data/validation/FSA_stress_based/case1_STRESINT.npy')

i = 41; j = 47; k = 6; t = 0 # a fault cell for fault 6

fault_strike = 180.96
fault_dip = 90
SH_azi = 292.13
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip

sigma, tau = StressTransform3D(0, SH[i-1,j-1,k-6,t], Sh[i-1,j-1,k-6,t], Sv[i-1,j-1,k-6,t], phi, theta)
fault_slip = ((tau - cohesion) / sigma >= mu).astype(np.int8)
print(fault_slip)

FSA = np.load('data/validation/FSA_stress_based/case1_FSA.npy')
print(FSA[i-1,j-1,k-6,t])

0
0.0


In [17]:
i = 40; j = 82; k = 6; t = 0 # a fault cell for fault 4

fault_strike = 75.69
fault_dip = 90
SH_azi = 292.13
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip

sigma, tau = StressTransform3D(0, SH[i-1,j-1,k-6,t], Sh[i-1,j-1,k-6,t], Sv[i-1,j-1,k-6,t], phi, theta)
fault_slip = ((tau - cohesion) / sigma >= mu).astype(np.int8)
print(fault_slip)

FSA = np.load('data/validation/FSA_stress_based/case1_FSA.npy')
print(FSA[i-1,j-1,k-6,t])

0
0.0


In [18]:
i = 40; j = 82; k = 6; t = 1 # a fault cell for fault 4

fault_strike = 75.69
fault_dip = 90
SH_azi = 292.13
mu = 0.6 #coefficient of friction
cohesion = 1 # fault cohesion in MPa

# compute rotation angles
phi = SH_azi - fault_strike
theta = fault_dip

sigma, tau = StressTransform3D(0, SH[i-1,j-1,k-6,t], Sh[i-1,j-1,k-6,t], Sv[i-1,j-1,k-6,t], phi, theta)
fault_slip = ((tau - cohesion) / sigma >= mu).astype(np.int8)
print(fault_slip)

FSA = np.load('data/validation/FSA_stress_based/case1_FSA.npy')
print(FSA[i-1,j-1,k-6,t])

1
1.0


In [ ]:
# check fault id
coor_fault = np.load('data/coor_fault/JD_Sula_2025_gmc_coor&fault_reservoir.npy')
coor_fault[i-1,j-1,k-6,3]

4.0